In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function

import datetime
import pandas as pd
import numpy as np
from dateutil import parser as dtparser

from utils.datafetch import *
from utils.vectorized_funs import *
from utils.datapipe import *
from utils.kerasutil import *
from utils import scenarioa
from FintechCapstone import FinCapstone

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

Using TensorFlow backend.


In [47]:
%matplotlib inline

plt.style.use('seaborn-whitegrid')
date_from = '2009-01-01'
date_to = '2017-03-01'
timespan = {
    "short_term": [1, 5]
    ,"medium_term": [40]
    ,"long_term": [90]
}

trial = FinCapstone(ticker_list=["GLW","HIMX","GLUU"], scenario="scenarioa", ticker_list_samplesize=4, timespan=timespan, date_from=date_from, reset_status=True)
#trial = FinCapstone(ticker_list=["ZHNE","BSFT","AERO","MATR","ITMSF","FIT","TIPT","OLBK","QADA","ATTU","LGF","STX","TPCS","GSAT","ZNGA","GLW","HIMX","YHOO", "NWSA","JAKK","GLUU","AMZN", "AAPL", "EBAY", "GOOG", "DIS", "NFLX", "EA", "TWTR", "FB", "TTWO", "PXLW", "UBI"], ticker_list_samplesize=4, timespan=timespan, timespan_ab=timespan_ab, date_from=date_from)
#trial = FinCapstone(ticker_list_samplesize=200, timespan=timespan, timespan_ab=timespan_ab, date_from=date_from)

#trial.run_initial_dataload()
#trial.feature_engineering()

In [44]:
~(trial.featureengineer_status_df["status"] == "NOK")

ticker
GLW      True
HIMX     True
GLUU    False
Name: status, dtype: bool

In [46]:
trial.provision_validtickerlist()

,status,start,end,msg
ticker,,,,
GLW,OK,2017-04-24 20:45:00.651392,2017-04-24 20:45:02.302525,NaN
HIMX,OK,2017-04-24 20:45:02.302684,2017-04-24 20:45:03.082218,NaN


In [7]:
X = trial.load_scenarioa_features("GLUU", True)
y = trial.load_scenarioa_labels("GLUU", True)

In [8]:
X.head()

,Date,Close,SMA_5,SMA_30,SMA_60,SMA_200,BOLL_5_UP,BOLL_5_DOWN,BOLL_30_UP,BOLL_30_DOWN,...,STOCOSCILATOR_14,STOCOSCILATOR_14_SMA,ADX,ADX_PDI,ADX_NDI,AROONUP_20,AROONDOWN_20,CHAIKIN_MFLOW_21,DAILY_MFLOW_21,OBV
0,2009-01-02,0.49,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.99992,0.571531,0.554785,0.0,0.0,0.506242,0.423777,-114693.877551
1,2009-01-05,0.50,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.99992,0.571531,0.554785,0.0,0.0,0.506242,0.424939,-72000.000000
2,2009-01-06,0.76,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.99992,0.571531,0.554785,0.0,0.0,0.506242,0.428664,210394.736842
3,2009-01-07,0.70,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.99992,0.571531,0.554785,0.0,0.0,0.506242,0.423576,80428.571429
4,2009-01-08,0.70,0.9,0.0,0.0,0.0,1.259137,0.540863,0.0,0.0,...,0.0,0.0,0.99992,0.571531,0.554785,0.0,0.0,0.506242,0.424338,122000.000000


In [9]:
y.head()

,Date,RETURN_1,RETURN_30,RETURN_60,RETURN_200
0,2009-01-02,0.020408,0.020408,-0.020408,1.632653
1,2009-01-05,0.520000,0.080000,0.060000,1.380000
2,2009-01-06,-0.078947,-0.250000,-0.197368,0.342105
3,2009-01-07,0.000000,-0.228571,-0.057143,0.571429
4,2009-01-08,0.157143,-0.257143,0.042857,0.642857


In [10]:
features_df = trial.load_scenarioa_features("GLUU", parseDate=True)
features_df.set_index("Date", inplace=True)

labels_df = trial.load_scenarioa_labels("GLUU", parseDate=True)
labels_df.set_index("Date", inplace=True)

model = scenarioa.create_model(3)
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from)
X_train


<class 'pandas.core.panel.Panel'>
Dimensions: 1510 (items) x 29 (major_axis) x 3 (minor_axis)
Items axis: 2010-01-04 00:00:00 to 2015-12-31 00:00:00
Major_axis axis: Close to OBV
Minor_axis axis: GLUU to HIMX

In [11]:
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, True)

In [12]:
X_train

<class 'pandas.core.panel.Panel'>
Dimensions: 1510 (items) x 29 (major_axis) x 3 (minor_axis)
Items axis: 2010-01-04 00:00:00 to 2015-12-31 00:00:00
Major_axis axis: Close to OBV
Minor_axis axis: GLUU to HIMX

In [13]:
y_train.head()

,RETURN_1,RETURN_30,RETURN_60,RETURN_200
Date,,,,
2010-01-04,0.049587,-0.132231,-0.190083,0.363636
2010-01-05,0.000000,-0.181102,-0.212598,0.322835
2010-01-06,0.031496,-0.188976,-0.212598,0.393701
2010-01-07,0.106870,-0.259542,-0.206107,0.343511
2010-01-08,-0.013793,-0.365517,-0.275862,0.151724


In [14]:
X_test

<class 'pandas.core.panel.Panel'>
Dimensions: 328 (items) x 29 (major_axis) x 3 (minor_axis)
Items axis: 2016-01-04 00:00:00 to 2017-04-21 00:00:00
Major_axis axis: Close to OBV
Minor_axis axis: GLUU to HIMX

In [15]:
y_test.head()

,RETURN_1,RETURN_30,RETURN_60,RETURN_200
Date,,,,
2016-01-04,-0.012397,0.479339,0.165289,-0.148760
2016-01-05,-0.020921,0.564854,0.171548,-0.142259
2016-01-06,-0.064103,0.666667,0.153846,-0.136752
2016-01-07,0.100457,0.726027,0.365297,-0.082192
2016-01-08,0.020747,0.539419,0.224066,-0.149378


In [16]:
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, "numpy")

In [17]:
X_train.shape

(1510, 29, 3)

In [18]:
y_train.shape

(1510, 4)

In [19]:
X_test.shape

(328, 29, 3)

In [20]:
y_test.shape

(328, 4)

In [21]:
model = scenarioa.create_model(len(trial.valid_ticker_list()))

In [22]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_8 (Convolution2D)  (None, 64, 27, 1)     640         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
zeropadding2d_7 (ZeroPadding2D)  (None, 64, 29, 3)     0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 64, 27, 1)     36928       zeropadding2d_7[0][0]            
____________________________________________________________________________________________________
zeropadding2d_8 (ZeroPadding2D)  (None, 64, 29, 3)     0           convolution2d_9[0][0]            
___________________________________________________________________________________________

In [25]:
model = scenarioa.create_model(len(trial.valid_ticker_list()))
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, True, "numpy")
scenarioa.fit(model, X_train, y_train)

Epoch 1/1
1510/1510 [==============================] - 1s - loss: 180.2744     
